## Preprocess

In [18]:
import os
from PIL import Image
def rotate_enhance(data_path):
    for class_ in os.listdir(data_path):
        dir_path = data_path + '/%s'%class_
        image_list = os.listdir(dir_path)
        print("目前處理Class：%s"%class_)
        for item in image_list:
            img = Image.open(dir_path + "/%s"%item)
            angle = 0
            op = [Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270]
            for i in op:
                new_img = img.transpose(i)
                new_img.save(dir_path +"/%s_%d.jpg"%(item,angle))
                new_img = new_img.transpose(Image.FLIP_LEFT_RIGHT)
                new_img.save(dir_path +"/%s_copy.jpg"%item)
                angle += 90


In [19]:
rotate_enhance("./data/train")

目前處理Class：0
目前處理Class：1


## Train

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 29 12:24:13 2022

@author: Kyle
"""

import os

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from utils import read_split_data, train_one_epoch,evaluate
from my_dataset import MyDataSet
from model import swin_large_patch4_window12_384_in22k

if __name__ == '__main__':
    
    num_classes = 2
    epochs = 100
    batch_size = 2
    lr = 2e-6
    img_size = 384
    
    data_path = "./data/train"
    weights = './pretrained/swin_large_patch4_window12_384_22k.pth'
    device = 'cuda:0'
    
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    
    if os.path.exists("./weights") is False:
        os.makedirs("./weights")
    
    
    train_images_path, train_images_label, val_images_path, val_images_label = read_split_data(data_path)
    
    data_transform = {
        "train": transforms.Compose([transforms.Resize([img_size,img_size]),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
        "val": transforms.Compose([transforms.Resize([img_size,img_size]),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}
    
    train_dataset = MyDataSet(images_path=train_images_path,
                              images_class=train_images_label,
                              transform=data_transform["train"])
    
    val_dataset = MyDataSet(images_path=val_images_path,
                            images_class=val_images_label,
                            transform=data_transform["val"])
    
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True,
                                               pin_memory=True,
                                               num_workers=nw,
                                               collate_fn=train_dataset.collate_fn)
    
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             pin_memory=True,
                                             num_workers=nw,
                                             collate_fn=val_dataset.collate_fn)
    
    model = swin_large_patch4_window12_384_in22k(num_classes=num_classes).to(device)
    
    if weights != "":
        assert os.path.exists(weights), "weights file: '{}' not exist.".format(weights)
        weights_dict = torch.load(weights, map_location=device)["model"]
        print("載入pretrained")
        for k in list(weights_dict.keys()):
            if "head" in k:
                del weights_dict[k]
        print(model.load_state_dict(weights_dict, strict=False))
    
    
    pg = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(pg, lr=lr, weight_decay=1e-8)
    
    for epoch in range(epochs):
        # train
        train_loss, train_acc = train_one_epoch(model=model,
                                                optimizer=optimizer,
                                                data_loader=train_loader,
                                                device=device,
                                                epoch=epoch)
    
        # validate
        '''
        val_loss, val_acc = evaluate(model=model,
                                     data_loader=val_loader,
                                     device=device,
                                     epoch=epoch)
        '''
    
        torch.save(model.state_dict(), "./weights/model_{}.pth".format(epoch))




4253 images were found in the dataset.
4253 images for training.
0 images for validation.
Using 2 dataloader workers every process
載入pretrained
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['layers.0.blocks.1.attn_mask', 'layers.1.blocks.1.attn_mask', 'layers.2.blocks.1.attn_mask', 'layers.2.blocks.3.attn_mask', 'layers.2.blocks.5.attn_mask', 'layers.2.blocks.7.attn_mask', 'layers.2.blocks.9.attn_mask', 'layers.2.blocks.11.attn_mask', 'layers.2.blocks.13.attn_mask', 'layers.2.blocks.15.attn_mask', 'layers.2.blocks.17.attn_mask'])
[train epoch 0] loss: 0.662, acc: 0.614:   1%|▍                                      | 22/2127 [00:11<11:25,  3.07it/s]

## Predict
分別得到每個model的結果及softmax前的結果用於ensemble

In [14]:
import os
import json

import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from model import swin_large_patch4_window12_384_in22k as create_model

os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"


def estimate(img,predict,df,map_,real):
    label = df[df['img']==img]['label'].values[0]
    if predict == label:
        #print("Correct\n")
        map_[str(label)] +=1
        real += 1
    #else:
        #print("Wrong,predict:%s but correct label is : %s"%(predict,label))
    return map_,real
def get_class_len(df,class_indict,num_classes):
    class_len =[]
    for i in range (num_classes):
        class_len.append(len(df[df['label'] == class_indict[str(i)]]))
    return class_len

def class_accuracy(class_len,map_):
    class_accuracy_map = map_.copy()
    i=0
    for item in class_accuracy_map:
        class_accuracy_map[item] = "%.2f"%((map_[item]/class_len[i])*100)
        i+=1
    return class_accuracy_map

def test():
    
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"using {device} device.")

    num_classes = 219
    img_size = 384
    data_transform = transforms.Compose([transforms.Resize([img_size,img_size]),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
        )

    test_path = "./data/test"
    json_path = './class_indices.json'
    assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)
    
    json_file = open(json_path, "r")
    class_indict = json.load(json_file)

    model = create_model(num_classes=num_classes).to(device)
    
    list_= ['55','60','65','75'] #想predict的model代號 (若命名方式不同 下面的model name 也要改)
    columns_ = ['img']
    for i in range(num_classes):
        columns_.append(str(i))
    for item in list_:
        print("\nCurrent is model : %s"%item)
        model_name = 'model_%s'%item
        ensemble_name = 'swin_384_%s'%model_name
    
        # load model weights
        model_weight_path = "./weights/%s.pth"%model_name#./weights/best_model.pth
        model.load_state_dict(torch.load(model_weight_path, map_location=device))
        model.eval()
        model.cuda()

        
        ensemble = pd.DataFrame(columns=columns_)
        result = pd.DataFrame(columns=['filename','category'])
        
        i=0
        f = os.listdir(test_path)
        print("共%d筆資料"%len(f))
        for x in f:
            img_path = test_path + "/" + x
            assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
            img = Image.open(img_path)
            #plt.imshow(img)
            # [N, C, H, W]
            img = data_transform(img)
            # expand batch dimension
            img = torch.unsqueeze(img, dim=0)
        
            # read class_indict
            # create model
    
            with torch.no_grad():
                # predict class
                output = torch.squeeze(model(img.to(device))).cpu()
                predict = torch.softmax(output, dim=0)
                predict_cla = torch.argmax(predict).numpy()
                
            #print("img:%s,predict:%s"%(x,class_indict[str(predict_cla)]))
            
            ensemble_output = output.numpy()
            dict_ = {'img':x}

            for y in range (num_classes):
                dict_[str(y)] = ensemble_output[y]
            
            ensemble = ensemble.append(dict_,ignore_index=True)
            #result = result.append({'filename':x,'category':class_indict[str(predict_cla)]},
            #                   ignore_index=True)
            #map_,real = estimate(x,class_indict[str(predict_cla)],df,map_,real)
            i+=1
            print("\rFinished:%d"%i,end='')
    
    
        #result.to_csv("./result/%s_result.csv"%ensemble_name,index=False)
        ensemble.to_csv("./result/%s_ens.csv"%ensemble_name,index=False)
    '''
    with open('./result/class_acc.json', 'w') as f:
        json.dump(class_accuracy_map, f)
    print(class_accuracy_map)
    '''
if __name__ == '__main__':
    test()


using cuda:0 device.

Current is model : 55
共24筆資料
Finished:24
Current is model : 60
共24筆資料
Finished:24
Current is model : 65
共24筆資料
Finished:24
Current is model : 75
共24筆資料
Finished:24

## Muti-Model Ensemble
ensemble_list：要合併的結果(softmax前)

ensemble_name：輸出的檔名

In [11]:
import pandas as pd
import torch
import json

import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"


ensemble_list = ['./result/swin_384_model_60_ens.csv',
                 './result/swin_384_model_55_ens.csv']

num_classes = 219
ensemble_name = 'swin50_55_60_65'

columns_ = ['img']
for i in range(num_classes):
    columns_.append(str(i))
    
result = pd.read_csv(ensemble_list[0],index_col="img")
result = result.sort_index(axis=0)

json_path = './class_indices.json'
json_file = open(json_path, "r")
class_indict = json.load(json_file)


for z,item in enumerate(ensemble_list):
    if z==0:
        continue
    df = pd.read_csv(item,index_col="img")
    df = df.sort_index(axis=0)
    if (list(df.index.values) == list(result.index.values)):
        result = result.add(df)
    else:
        print("%d index error")%z

#result = result.div(len(ensemble_list))
i=0
data = pd.DataFrame(columns=['filename','category'])
for item in list(result.index.values):
    
    raw = torch.from_numpy(result.loc[item].values)
    output = torch.squeeze(raw).cpu()
    predict = torch.softmax(output, dim=0)

    predict_cla = torch.argmax(predict).numpy()
    i+=1
    print("\r已完成：%d張"%i,end='')

    data = data.append({'filename':item,'category':class_indict[str(predict_cla)]},
                      ignore_index=True)


data.to_csv("./result/%s_result.csv"%ensemble_name,index=False)
print("\nSuccess Ensemble.")
print("結果輸出到./result")

已完成：24張
Success Ensemble.
結果輸出到./result
